In [987]:
import time

# Problem 1
## Pins

In [1014]:
# Good luck!
d = {
    '1': ['1', '2', '4'],
    '2': ['1', '2', '3', '5'],
    '3': ['2', '3', '6'],
    '4': ['1', '4', '5', '7'],
    '5': ['2', '4', '5', '6', '8'],
    '6': ['3', '5', '6', '9'],
    '7': ['4', '7', '8'],
    '8': ['0', '5', '7', '8', '9'],
    '9': ['6', '8', '9'],
    '0': ['0', '8']
}

pin = input()
length = len(pin)

full_length = 1
for digit in pin:
    full_length *= len(d[digit])
    
pins = []
for digit in pin:
    if not pins:
        dim = full_length / len(d[digit])
        for prob_digit in d[digit]:
            pins.extend(int(dim)*[prob_digit])
        repeating = len(d[digit])
        last_ = len(d[digit])
    else:
        dim /= len(d[digit])

        new_digits = []
        for prob_digit in d[digit]:
            template = int(dim) * [prob_digit]
            new_digits.extend(template)
        new_digits *= repeating
        repeating *= len(d[digit])
        
        for i in range(repeating):
            for j in range(int(dim)):
                pins[int(full_length/repeating*i + j)] += new_digits[int(full_length/repeating*i + j)]
print(",".join(pins))

76
43,45,46,49,73,75,76,79,83,85,86,89


# Problem 2
## Skyscrappers

In [1005]:
###########################################################
# ----- Bunch of functions to remove n from the field -----
###########################################################

# ----- Function to remove n from the whole row/col -----
def del_from_state(state, n, row=None, col=None):
    if row is not None:
        for i in range(6):
            if len(state[row][i]) > 1:
                state[row][i] -= n
    elif col is not None:
        for i in range(6):
            if len(state[i][col]) > 1:
                state[i][col] -= n
    else:
        print('Enter col or row')
        raise ValueError
    return state

# ----- Function to remove all values in the one cell except n -----
# Comment: only one cell in row/column consist of n
def del_inapp_numbers(state, n, row=None, col=None):
    if row is not None:
        for i in range(6):
            if n in state[row][i]:
                state[row][i] = set([n])
                return state
    elif col is not None:
        for i in range(6):
            if n in state[i][col]:
                state[i][col] = set([n])
                return state
    else:
        print('Enter col or row')
        raise ValueError
    return state

# ----- If we have cell with determined choise, we will remove this n from other cells in row/col -----
# E.g. in row 0: [{3}, {3,4}, {6,6}, {3,4}, {1,2}, {4,5}]
# We understand that in there is no need in 3 in other cells except [0][1]
def del_singles(state, row=None, col=None):
    if row is not None:
        # Go from 0 to 5
        for i in range(6):
            if len(state[row][i]) == 1:
                state = del_from_state(state, state[row][i], row=row)
        # Go from 5 to 0
        for i in range(6)[::-1]:
            if len(state[row][i]) == 1:
                state = del_from_state(state, state[row][i], row=row)
    elif col is not None:
        # Go from 0 to 5
        for i in range(6):
            if len(state[i][col]) == 1:
                state = del_from_state(state, state[i][col], col=col)
        # Go from 5 to 0
        for i in range(6)[::-1]:
            if len(state[i][col]) == 1:
                state = del_from_state(state, state[i][col], col=col)
    else:
        print('Enter col or row')
        raise ValueError
    return state

# ----- This function check if we can drop some digits because of there is only one choice -----
# E.g. in row 0: [{1,2,3}, {3,4}, {5,6}, {3,4}, {1,2}, {4,5}]
# We understand that in the cell [0][2] should be 6.
# With this function we will remove all other digits from the cell [0][2]
def check_uniqueness(state, row=None, col=None):
    d = {i: 0 for i in range(1,7)}
    
    if row is not None:
        for i in range(6):
            for n in state[row][i]:
                d[n] += 1
        for n in d:
            if d[n] == 1:
                state = del_inapp_numbers(state, n, row=row)
                
    elif col is not None:
        for i in range(6):
            for n in state[i][col]:
                d[n] += 1
        for n in d:
            if d[n] == 1:
                state = del_inapp_numbers(state, n, col=col)
    else:
        print('Enter col or row')
        raise ValueError
    return state

# ----- Complex functions for checking the filed -----
def reduce_from_row(state, row):
    state = del_singles(state, row=row)
    state = check_uniqueness(state, row=row)
    return state

def reduce_from_col(state, col):
    state = del_singles(state, col=col)
    state = check_uniqueness(state, col=col)
    return state

##################################################################
# ----- Bunch of functions to work with array of constraints -----
##################################################################

# ----- Return position of constraint -----
def pos_of_constr(i):
    if i // 6 == 0:
        return -1, i % 6
    elif i // 6 == 1:
        return i % 6, 6
    elif i // 6 == 2:
        return 6, 5 - i%6
    elif i // 6 == 3:
        return 5 - i%6, -1
    else:
        print('Bad position of constraint')

# ----- Drop forbidden digits from the edge there is a constraint -----
def drop_from_constr(state, row, col, c):
    if row == -1:
        for row_c in range(c-1):
            state[row_c][col] -= set(range(1,7)[-(c-row_c-1):])
        return state
    elif row == 6:
        for row_c in range(c-1):
            state[5-row_c][col] -= set(range(1,7)[-(c-row_c-1):])
        return state
    elif col == -1:
        for col_c in range(c-1):
            state[row][col_c] -= set(range(1,7)[-(c-col_c-1):])
        return state
    elif col == 6:
        for col_c in range(c-1):
            state[row][5-col_c] -= set(range(1,7)[-(c-col_c-1):])
        return state
    
# ----- Drop forbidden digits from the all edges -----
def reduce_from_constr(state, constraints):
    for i, c in enumerate(constraints):
        if c:
            row, col = pos_of_constr(i)
            state = drop_from_constr(state, row, col, c)
    return state

##################################################################
# ----- Bunch of functions to check the field for conditions -----
##################################################################

# ----- Check that constraint for column is fullfiled (n buildings are observed) -----
def check_col(state, col, cond, top=True):
    vis_build = 1
    shift = 0 if top else 1
    for i1 in state[abs(0-shift*5)][col]:
        for i2 in state[abs(1-shift*5)][col]:
            if i2 == i1:
                continue
            for i3 in state[abs(2-shift*5)][col]:
                if i3 == i2 or i3 == i1:
                    continue
                for i4 in state[abs(3-shift*5)][col]:
                    if i4 == i3 or i4 == i2 or i4 == i1:
                        continue
                    for i5 in state[abs(4-shift*5)][col]:
                        if i5 == i4 or i5 == i3 or i5 == i2 or i5 == i1:
                            continue
                        for i6 in state[abs(5-shift*5)][col]:
                            if i6 == i5 or i6 == i4 or i6 == i3 or i6 == i2 or i6 == i1:
                                continue
                            prev_max = i1
                            for next_build in [i2, i3, i4, i5, i6]:
                                if next_build > prev_max:
                                    vis_build += 1
                                    prev_max = next_build
                            if vis_build == cond:
                                return True
                            else:
                                vis_build = 1
    return False

# ----- Check that constraint for row is fullfiled (n buildings are observed) -----
def check_row(state, row, cond, left=True):
    vis_build = 1
    shift = 0 if left else 1
    for i1 in state[row][abs(0-shift*5)]:
        for i2 in state[row][abs(1-shift*5)]:
            if i2 == i1:
                continue
            for i3 in state[row][abs(2-shift*5)]:
                if i3 == i2 or i3 == i1:
                    continue
                for i4 in state[row][abs(3-shift*5)]:
                    if i4 == i3 or i4 == i2 or i4 == i1:
                        continue
                    for i5 in state[row][abs(4-shift*5)]:
                        if i5 == i4 or i5 == i3 or i5 == i2 or i5 == i1:
                            continue
                        for i6 in state[row][abs(5-shift*5)]:
                            if i6 == i5 or i6 == i4 or i6 == i3 or i6 == i2 or i6 == i1:
                                continue
                            prev_max = i1
                            for next_build in [i2, i3, i4, i5, i6]:
                                if next_build > prev_max:
                                    vis_build += 1
                                    prev_max = next_build
                            if vis_build == cond:
                                return True
                            else:
                                vis_build = 1
    return False

# ----- Check that there is not two the same digits in the row/col -----
def check_ones(row):
    ones = []
    for i,s in enumerate(row):
        if len(s) == 1:
            ones.append(i)
    if len(ones) <= 1:
        return True
    for i in ones[:-1]:
        j = i + 1
        while j <= ones[-1]:
            if row[i] == row[j]:
                return False
            j += 1
    return True

# ----- Check upper function for the whole field -----
def check_state_for_ones(state):
    for i in range(6):
        if not check_ones(state[i]):
#             print('Uniqueness is broken for row=%d' % i)
            return False
        if not check_ones(state[:][i]):
#             print('Uniqueness is broken for col=%d' % i)
            return False
    return True
    
# ----- Check the field that all constraints and conditions are fullfilled -----    
def check_constr(state, constraints):
    for i, c in enumerate(constraints):
        if c:
            row, col = pos_of_constr(i)
            if row == -1:
                if check_col(state, col, c, top=True):
                    continue
                else:
#                     print('Constraint for row=%d, col=%d is broken' % (row, col))
                    return False
            if row == 6:
                if check_col(state, col, c, top=False):
                    continue
                else:
#                     print('Constraint for row=%d, col=%d is broken' % (row, col))
                    return False
            if col == -1:
                if check_row(state, row, c, left=True):
                    continue
                else:
#                     print('Constraint for row=%d, col=%d is broken' % (row, col))
                    return False
            if col == 6:
                if check_row(state, row, c, left=False):
                    continue
                else:
#                     print('Constraint for row=%d, col=%d is broken' % (row, col))
                    return False
    if not check_state_for_ones(state):
        return False
    
    return True

# ----- Check that there is only one possible digit for each cell -----
def check_done(state):
    for row in range(6):
        for col in range(6):
            if len(state[row][col]) != 1:
                return False
    return True

# ----- Check if the field is good or not -----
def guess(state, constraints):
    # Save initial state
    for tries in range(5):
        for i in range(6):
            state = reduce_from_row(state, i)
            state = reduce_from_col(state, i)
    if check_constr(state, constraints) and not check_done(state):
        return 'Need one more guess', False
    elif check_constr(state, constraints) and check_done(state):
        return state, True
    elif not check_constr(state, constraints):
        return 'Try another guess', False
    
# ----- Function to create a copy of 2D array with sets -----
def copy_state(state):
    old_state = [[[] for _ in range(6)] for _ in range(6)]
    for i in range(6):
        for j in range(6):
            for n in state[i][j]:
                old_state[i][j].append(n)
    for i in range(6):
        for j in range(6):
            old_state[i][j] = set(old_state[i][j])
    return old_state
    
# ----- Function that try to guess another digit... -----
# ...after all other variants already are ended up
def try_row_col(state, constraints, row, col):
    old_state_row_col = []
    for n in state[row][col]:
        old_state_row_col.append(n)
    
    old_state = copy_state(state)
    for n in old_state_row_col:
        if check_constr(state, constraints) and check_done(state):
            return state
        else:
            state = old_state.copy()
            old_state = copy_state(state)
        state[row][col] = set([n])
        msg, res = guess(state, constraints)
        if msg == 'Try another guess':
            state = old_state.copy()
            old_state = copy_state(state)
            continue
        elif res:
            state = msg
            if check_constr(state, constraints) and check_done(state):
                break
        elif msg == 'Need one more guess':
            if col < 5 and row <= 5:
                col_new = col + 1
                state = try_row_col(state, constraints, row, col_new)
                if check_constr(state, constraints) and check_done(state):
                    return state
                    break
            elif col == 5 and row <= 5:
                row_new = row + 1
                col_new = 0
                state = try_row_col(state, constraints, row_new, col_new)
                if check_constr(state, constraints) and check_done(state):
                    return state
                    break
            elif row > 5:
                print('Cant find solution')
    return state                

In [1006]:
# Test 1
constraints = [3,2,2,3,2,1,1,2,3,3,2,2,5,1,2,2,4,3,3,2,1,2,2,4]

In [1010]:
start = time.time()
# Set new state
state = [[1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6]]
for row in range(6):
    for col in range(6):
        state[row][col] = set([1,2,3,4,5,6])

# Stare remove possibilities
for i in range(10):
    state_old = state
    state = reduce_from_constr(state, constraints)
    for i in range(6):
        state = reduce_from_row(state, i)
        state = reduce_from_col(state, i)
        

end = time.time()
state = try_row_col(state, constraints, 0, 1)

res = []
for i in range(6):
    for j in range(6):
        res.append(str(list(state[i][j])[0]))
print(",".join(res))
print('Exec time=%2.4f sec' % (end-start))

2,1,4,3,5,6,1,6,3,2,4,5,4,3,6,5,1,2,6,5,2,1,3,4,5,4,1,6,2,3,3,2,5,4,6,1
Exec time=0.0024 sec


In [1011]:
# Test 2
constraints = [0,0,0,2,2,0,0,0,0,6,3,0,0,4,0,0,0,0,4,4,0,3,0,0]

In [1012]:
start = time.time()
# Set new state
state = [[1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6],
         [1, 2, 3, 4, 5, 6]]
for row in range(6):
    for col in range(6):
        state[row][col] = set([1,2,3,4,5,6])

# Stare remove possibilities
for i in range(10):
    state_old = state
    state = reduce_from_constr(state, constraints)
    for i in range(6):
        state = reduce_from_row(state, i)
        state = reduce_from_col(state, i)
        

end = time.time()
state = try_row_col(state, constraints, 0, 1)

res = []
for i in range(6):
    for j in range(6):
        res.append(str(list(state[i][j])[0]))
print(",".join(res))
print('Exec time=%2.4f sec' % (end-start))

5,6,1,4,3,2,4,1,3,2,6,5,2,3,6,1,5,4,6,5,4,3,2,1,1,2,5,6,4,3,3,4,2,5,1,6
Exec time=0.0026 sec
